In [39]:
import tensorflow as ft
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [40]:
#Loading an train model
model=load_model('model.h5')

#Load the encoder and scaler
with open('Fscaler.pkl','rb') as file:
    Fscaler=pickle.load(file)

with open('label_enocoder_gender.pkl','rb') as file:
    label_encoder=pickle.load(file)

with open('onehotEncoder.pkl','rb') as file:
    onehotencoder=pickle.load(file)

In [41]:
#EXample input data
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [42]:
data=pd.DataFrame([input_data])

In [43]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [44]:
data['Gender']=label_encoder.transform((data['Gender'],))[0]

C:\Users\vishn\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [45]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [46]:
a=onehotencoder.transform([data['Geography']]).toarray()
gro_encoded_df=pd.DataFrame(data=a,columns=onehotencoder.get_feature_names_out(['Geography']))
gro_encoded_df

C:\Users\vishn\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [47]:
u=data.iloc[:,0]
h=pd.concat([u,gro_encoded_df],axis=1)
data=pd.concat([h,data.iloc[:,2:]],axis=1)
data

,CreditScore,Geography_France,Geography_Germany,Geography_Spain,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,1.0,0.0,0.0,1,40,3,60000,2,1,1,50000


In [48]:
data=Fscaler.transform(data)
data

array([[-0.52493139,  0.99451504, -0.57658047, -0.57388614,  0.91255717,
         0.09471269, -0.69884427, -0.26134615,  0.79931788,  0.64007158,
         0.97530483, -0.87196872]])

In [49]:
#Prediction 
prediction=model.predict(data)
pred_prob=prediction[0][0]
pred_prob

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


0.018441785

In [50]:
if pred_prob>0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
